In [2]:
%pip install jinja2

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)

   -------------------- ------------------- 1/2 [jinja2]
   -------------------- ------------------- 1/2 [jinja2]
   -------------------- ------------------- 1/2 [jinja2]
   ---------------------------------------- 2/2 [jinja2]

Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Set style parameters
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("colorblind")

# IEEE colors
IEEE_BLUE = '#00629B'
IEEE_LIGHTBLUE = '#99C2E8'
IEEE_GREY = '#A9A9A9'
IEEE_DARKBLUE = '#003F63'

def load_metrics_for_comparison(platform, metric_name, scenario):
    """
    Load metrics from the platform's output directory for comparison
    
    Args:
        platform: Name of the platform (JADE, SPADE)
        metric_name: Name of the metric (capacity, compactness, occupancy, time_eligibility, RE)
        scenario: Scenario name (full, medium, small)
        
    Returns:
        Loaded metrics data or None if not found
    """
    try:
        # Construct the path to the metrics file
        if metric_name == 'RE':
            # RE metrics are stored differently
            file_path = os.path.join(
                "../scheduling_output", 
                platform, 
                metric_name, 
                "all", 
                f"{platform}_RE_analysis.json"
            )
            
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Extract scenario-specific data
                return data['scenarios'][scenario]
        else:
            # Regular metrics stored in scenario-specific folders
            metric_filename = {
                'capacity': 'metricas_capacidad.json',
                'compactness': 'metricas_compactacion.json',
                'occupancy': 'metricas_ocupacion.json',
                'time_eligibility': 'metricas_te.json'
            }
            
            file_path = os.path.join(
                "../scheduling_output", 
                platform, 
                metric_name, 
                scenario, 
                metric_filename[metric_name]
            )
            
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
                
    except (FileNotFoundError, json.JSONDecodeError, KeyError) as e:
        print(f"Error loading {metric_name} metrics for {platform}, {scenario}: {str(e)}")
        return None

def create_comparison_dataframe(metric_name, platforms=['JADE', 'SPADE'], scenarios=['full', 'medium', 'small']):
    """
    Creates a DataFrame for cross-platform comparison of a specific metric
    
    Args:
        metric_name: The metric to compare
        platforms: List of platforms to compare
        scenarios: List of scenarios to include
        
    Returns:
        DataFrame with comparison data
    """
    comparison_data = []
    
    for scenario in scenarios:
        row = {'Scenario': scenario}
        
        for platform in platforms:
            # Load metrics for this platform and scenario
            metrics = load_metrics_for_comparison(platform, metric_name, scenario)
            
            if metrics is None:
                continue
                
            # Extract relevant metrics based on metric type
            if metric_name == 'capacity':
                # For capacity metrics, get the global stats
                row[f"{platform}_bajo_capacidad"] = metrics['global_stats']['bajo_capacidad']
                row[f"{platform}_capacidad_exacta"] = metrics['global_stats']['capacidad_exacta']
                row[f"{platform}_sobre_capacidad"] = metrics['global_stats']['sobre_capacidad']
                
                # Add room stats aggregation
                if 'room_stats' in metrics:
                    room_stats = metrics['room_stats']
                    total_rooms = len(room_stats)
                    row[f"{platform}_total_rooms"] = total_rooms
                
            elif metric_name == 'compactness':
                # For compactness, calculate average windows per room
                total_ventanas = sum(room['ventanas'] for room in metrics['estadisticas_por_sala'].values())
                total_rooms = len(metrics['estadisticas_por_sala'])
                row[f"{platform}_avg_ventanas"] = total_ventanas / total_rooms if total_rooms > 0 else 0
                row[f"{platform}_total_ventanas"] = total_ventanas
                row[f"{platform}_total_rooms"] = total_rooms
                
                # Calculate window distribution by duration
                window_by_duration = {}
                for room_stats in metrics['estadisticas_duracion'].values():
                    for duration, count in room_stats.get('duraciones', {}).items():
                        duration = int(duration)
                        if duration not in window_by_duration:
                            window_by_duration[duration] = 0
                        window_by_duration[duration] += count
                
                for duration, count in window_by_duration.items():
                    row[f"{platform}_ventanas_duracion_{duration}"] = count
                
            elif metric_name == 'occupancy':
                # For occupancy, get the global occupancy rate
                row[f"{platform}_tasa_ocupacion"] = metrics['global_stats']['tasa_ocupacion_global']
                row[f"{platform}_total_ocupados"] = metrics['global_stats']['total_ocupados']
                row[f"{platform}_total_desocupados"] = metrics['global_stats']['total_desocupados']
                
                # Add daily occupancy stats
                for day, day_stats in metrics.get('daily_stats', {}).items():
                    row[f"{platform}_ocupacion_{day}"] = day_stats.get('tasa_ocupacion', 0)
                
            elif metric_name == 'time_eligibility':
                # For time eligibility, get the TE percentage
                row[f"{platform}_te_percentage"] = metrics['te_percentage']
                row[f"{platform}_te_value"] = metrics['te_value']
                
            elif metric_name == 'RE':
                # For RE metrics, get the RE percentage and related metrics
                row[f"{platform}_re_percentage"] = metrics['re_percentage']
                row[f"{platform}_total_events"] = metrics['total_events']
                row[f"{platform}_total_rooms"] = metrics['total_rooms']
                
                # Add limited flexibility events count
                limited_flex_count = len(metrics.get('limited_flexibility_events', []))
                row[f"{platform}_limited_flex_events"] = limited_flex_count
                
                if 'schedule_quality' in metrics and metrics['schedule_quality']:
                    quality = metrics['schedule_quality']
                    row[f"{platform}_scheduling_rate"] = quality.get('scheduling_rate_percentage', 0)
                    row[f"{platform}_avg_satisfaction"] = quality.get('avg_satisfaction', 0)
                    row[f"{platform}_campus_match_rate"] = quality.get('campus_match_rate_percentage', 0)
                    row[f"{platform}_good_capacity_rate"] = quality.get('good_capacity_rate_percentage', 0)
        
        comparison_data.append(row)
    
    # Create DataFrame
    return pd.DataFrame(comparison_data)

def create_comparative_charts(metric_name, df, output_dir="../scheduling_output/comparative"):
    """
    Creates comparative charts for the specified metric
    
    Args:
        metric_name: The metric being compared
        df: DataFrame with comparison data
        output_dir: Directory to save output charts
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    if metric_name == 'capacity':
        # Create grouped bar chart for capacity metrics
        fig, ax = plt.subplots(figsize=(12, 8))
        
        x = np.arange(len(df['Scenario']))  # the label locations
        width = 0.15  # the width of the bars
        
        # Create bars for each platform and capacity type
        platforms = ['JADE', 'SPADE']
        metrics = ['bajo_capacidad', 'capacidad_exacta', 'sobre_capacidad']
        colors = [IEEE_LIGHTBLUE, IEEE_BLUE, IEEE_GREY]
        
        for i, platform in enumerate(platforms):
            for j, metric in enumerate(metrics):
                col = f"{platform}_{metric}"
                if col in df.columns:
                    offset = width * (i * len(metrics) + j - len(metrics))
                    ax.bar(x + offset, df[col], width, label=f"{platform} - {metric.replace('_', ' ').title()}", 
                           color=colors[j], alpha=0.7 if platform == 'SPADE' else 1.0)
        
        # Add labels and legend
        ax.set_ylabel('Porcentaje (%)')
        ax.set_title('Distribución de Capacidad: JADE vs SPADE')
        ax.set_xticks(x)
        ax.set_xticklabels([s.capitalize() for s in df['Scenario']])
        ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'capacity_comparison.png'), bbox_inches='tight', dpi=300)
        plt.close()
        
        # Create a pie chart comparison for optimal capacity usage
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
        
        # JADE Pie Chart
        jade_data = df[[c for c in df.columns if 'JADE_' in c and 'capacidad' in c]].iloc[0].values
        jade_labels = ['Bajo Capacidad', 'Capacidad Exacta', 'Sobre Capacidad']
        
        ax1.pie(jade_data, labels=jade_labels, colors=colors, autopct='%1.1f%%', 
                startangle=90, wedgeprops={'edgecolor': 'w'})
        ax1.set_title('JADE - Distribución de Capacidad (Escenario Completo)')
        
        # SPADE Pie Chart
        spade_data = df[[c for c in df.columns if 'SPADE_' in c and 'capacidad' in c]].iloc[0].values
        
        ax2.pie(spade_data, labels=jade_labels, colors=colors, autopct='%1.1f%%', 
                startangle=90, wedgeprops={'edgecolor': 'w'})
        ax2.set_title('SPADE - Distribución de Capacidad (Escenario Completo)')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'capacity_pie_comparison.png'), bbox_inches='tight', dpi=300)
        plt.close()
        
    elif metric_name == 'compactness':
        # Create bar chart for average windows per room
        plt.figure(figsize=(10, 6))
        
        jade_data = df['JADE_avg_ventanas'].values
        spade_data = df['SPADE_avg_ventanas'].values
        
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        plt.bar(x - width/2, jade_data, width, label='JADE', color=IEEE_BLUE)
        plt.bar(x + width/2, spade_data, width, label='SPADE', color=IEEE_GREY)
        
        plt.xlabel('Escenario')
        plt.ylabel('Ventanas Promedio por Sala')
        plt.title('Comparación de Compactación: JADE vs SPADE')
        plt.xticks(x, [s.capitalize() for s in df['Scenario']])
        plt.legend()
        
        # Add values on top of bars
        for i, v in enumerate(jade_data):
            plt.text(i - width/2, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold')
            
        for i, v in enumerate(spade_data):
            plt.text(i + width/2, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'compactness_comparison.png'), dpi=300)
        plt.close()
        
        # Create stacked bar chart for window durations
        duration_cols = [c for c in df.columns if 'ventanas_duracion_' in c]
        if duration_cols:
            for idx, row in df.iterrows():
                scenario = row['Scenario']
                
                # Create figure with two subplots side by side
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
                
                # Extract data for each platform
                platforms = ['JADE', 'SPADE']
                for i, platform in enumerate([ax1, ax2]):
                    platform_name = platforms[i]
                    durations = []
                    counts = []
                    
                    for col in df.columns:
                        if f'{platform_name}_ventanas_duracion_' in col and not pd.isna(row[col]):
                            duration = int(col.split('_')[-1])
                            durations.append(duration)
                            counts.append(row[col])
                    
                    # Sort by duration
                    sorted_data = sorted(zip(durations, counts))
                    if sorted_data:
                        durations, counts = zip(*sorted_data)
                        
                        # Create bar chart
                        bars = platform.bar(durations, counts, color=[IEEE_LIGHTBLUE, IEEE_BLUE, IEEE_DARKBLUE][:len(durations)])
                        platform.set_xlabel('Duración de Ventana (bloques)')
                        platform.set_ylabel('Número de Ventanas')
                        platform.set_title(f'{platform_name} - Distribución de Ventanas por Duración ({scenario.capitalize()})')
                        platform.set_xticks(durations)
                        
                        # Add value labels on bars
                        for bar in bars:
                            height = bar.get_height()
                            platform.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                                    f'{int(height)}', ha='center', va='bottom')
                
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, f'compactness_duration_{scenario}.png'), dpi=300)
                plt.close()
        
    elif metric_name == 'occupancy':
        # Create bar chart for occupancy rate
        plt.figure(figsize=(10, 6))
        
        jade_data = df['JADE_tasa_ocupacion'].values
        spade_data = df['SPADE_tasa_ocupacion'].values
        
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        plt.bar(x - width/2, jade_data, width, label='JADE', color=IEEE_BLUE)
        plt.bar(x + width/2, spade_data, width, label='SPADE', color=IEEE_GREY)
        
        plt.xlabel('Escenario')
        plt.ylabel('Tasa de Ocupación (%)')
        plt.title('Comparación de Ocupación de Salas: JADE vs SPADE')
        plt.xticks(x, [s.capitalize() for s in df['Scenario']])
        plt.legend()
        
        # Add values on top of bars
        for i, v in enumerate(jade_data):
            plt.text(i - width/2, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')
            
        for i, v in enumerate(spade_data):
            plt.text(i + width/2, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'occupancy_comparison.png'), dpi=300)
        plt.close()
        
        # Create stacked bar chart for occupied vs unoccupied blocks
        fig, axes = plt.subplots(len(df), 2, figsize=(15, 4*len(df)))
        if len(df) == 1:
            axes = np.array([axes])
            
        for idx, row in df.iterrows():
            scenario = row['Scenario']
            
            # Data for each platform
            jade_occupied = row.get('JADE_total_ocupados', 0)
            jade_unoccupied = row.get('JADE_total_desocupados', 0)
            jade_total = jade_occupied + jade_unoccupied
            
            spade_occupied = row.get('SPADE_total_ocupados', 0)
            spade_unoccupied = row.get('SPADE_total_desocupados', 0)
            spade_total = spade_occupied + spade_unoccupied
            
            # Create stacked bar chart for JADE
            axes[idx, 0].bar(['Ocupación'], [jade_occupied/jade_total*100], color=IEEE_BLUE, label='Bloques Ocupados')
            axes[idx, 0].bar(['Ocupación'], [jade_unoccupied/jade_total*100], bottom=[jade_occupied/jade_total*100], 
                    color=IEEE_GREY, label='Bloques Desocupados')
            axes[idx, 0].set_ylabel('Porcentaje (%)')
            axes[idx, 0].set_title(f'JADE - {scenario.capitalize()}')
            axes[idx, 0].legend()
            
            # Add percentage text
            axes[idx, 0].text(0, jade_occupied/jade_total*100/2, f'{jade_occupied/jade_total*100:.1f}%', 
                      ha='center', va='center', color='white', fontweight='bold')
            axes[idx, 0].text(0, jade_occupied/jade_total*100 + jade_unoccupied/jade_total*100/2, 
                      f'{jade_unoccupied/jade_total*100:.1f}%', ha='center', va='center', fontweight='bold')
            
            # Create stacked bar chart for SPADE
            axes[idx, 1].bar(['Ocupación'], [spade_occupied/spade_total*100], color=IEEE_BLUE, label='Bloques Ocupados')
            axes[idx, 1].bar(['Ocupación'], [spade_unoccupied/spade_total*100], bottom=[spade_occupied/spade_total*100], 
                    color=IEEE_GREY, label='Bloques Desocupados')
            axes[idx, 1].set_ylabel('Porcentaje (%)')
            axes[idx, 1].set_title(f'SPADE - {scenario.capitalize()}')
            axes[idx, 1].legend()
            
            # Add percentage text
            axes[idx, 1].text(0, spade_occupied/spade_total*100/2, f'{spade_occupied/spade_total*100:.1f}%', 
                      ha='center', va='center', color='white', fontweight='bold')
            axes[idx, 1].text(0, spade_occupied/spade_total*100 + spade_unoccupied/spade_total*100/2, 
                      f'{spade_unoccupied/spade_total*100:.1f}%', ha='center', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'occupancy_stacked_comparison.png'), dpi=300)
        plt.close()
        
        # Create line chart for day-by-day occupancy
        day_cols = [c for c in df.columns if 'ocupacion_' in c]
        if day_cols:
            for idx, row in df.iterrows():
                scenario = row['Scenario']
                
                # Extract days and values for both platforms
                days = []
                jade_values = []
                spade_values = []
                
                for col in df.columns:
                    if 'JADE_ocupacion_' in col and not pd.isna(row[col]):
                        day = col.split('_')[-1]
                        days.append(day)
                        jade_values.append(row[col])
                        
                        # Get corresponding SPADE value
                        spade_col = col.replace('JADE', 'SPADE')
                        spade_values.append(row.get(spade_col, 0))
                
                if days:
                    # Reorder days to standard week order
                    correct_order = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes']
                    ordered_indices = [days.index(day) for day in correct_order if day in days]
                    
                    ordered_days = [days[i] for i in ordered_indices]
                    ordered_jade = [jade_values[i] for i in ordered_indices]
                    ordered_spade = [spade_values[i] for i in ordered_indices]
                    
                    # Create line chart
                    plt.figure(figsize=(10, 6))
                    plt.plot(ordered_days, ordered_jade, 'o-', color=IEEE_BLUE, linewidth=2, markersize=8, label='JADE')
                    plt.plot(ordered_days, ordered_spade, 's-', color=IEEE_GREY, linewidth=2, markersize=8, label='SPADE')
                    
                    plt.xlabel('Día de la Semana')
                    plt.ylabel('Tasa de Ocupación (%)')
                    plt.title(f'Comparación de Ocupación Diaria: {scenario.capitalize()}')
                    plt.grid(True, linestyle='--', alpha=0.7)
                    plt.legend()
                    
                    # Add value labels
                    for i, v in enumerate(ordered_jade):
                        plt.text(i, v + 1, f'{v:.1f}%', ha='center', color=IEEE_BLUE)
                        
                    for i, v in enumerate(ordered_spade):
                        plt.text(i, v - 2, f'{v:.1f}%', ha='center', color=IEEE_GREY)
                    
                    plt.tight_layout()
                    plt.savefig(os.path.join(output_dir, f'occupancy_daily_{scenario}.png'), dpi=300)
                    plt.close()
        
    elif metric_name == 'time_eligibility':
        # Create bar chart for time eligibility
        plt.figure(figsize=(10, 6))
        
        jade_data = df['JADE_te_percentage'].values
        spade_data = df['SPADE_te_percentage'].values
        
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        plt.bar(x - width/2, jade_data, width, label='JADE', color=IEEE_BLUE)
        plt.bar(x + width/2, spade_data, width, label='SPADE', color=IEEE_GREY)
        
        plt.xlabel('Escenario')
        plt.ylabel('Elegibilidad de Time-slot (%)')
        plt.title('Comparación de Elegibilidad de Time-slot: JADE vs SPADE')
        plt.xticks(x, [s.capitalize() for s in df['Scenario']])
        plt.legend()
        
        # Add values on top of bars
        for i, v in enumerate(jade_data):
            plt.text(i - width/2, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')
            
        for i, v in enumerate(spade_data):
            plt.text(i + width/2, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'time_eligibility_comparison.png'), dpi=300)
        plt.close()
        
        # Create line chart showing TE trend across scenarios
        plt.figure(figsize=(10, 6))
        
        scenarios = [s.capitalize() for s in df['Scenario']]
        
        plt.plot(scenarios, jade_data, 'o-', color=IEEE_BLUE, linewidth=2, markersize=8, label='JADE')
        plt.plot(scenarios, spade_data, 's-', color=IEEE_GREY, linewidth=2, markersize=8, label='SPADE')
        
        plt.xlabel('Escenario')
        plt.ylabel('Elegibilidad de Time-slot (%)')
        plt.title('Tendencia de Elegibilidad de Time-slot')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        
        # Add value labels
        for i, v in enumerate(jade_data):
            plt.text(i, v + 0.3, f'{v:.2f}%', ha='center', color=IEEE_BLUE)
            
        for i, v in enumerate(spade_data):
            plt.text(i, v - 0.3, f'{v:.2f}%', ha='center', color=IEEE_GREY)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'time_eligibility_trend.png'), dpi=300)
        plt.close()
        
    elif metric_name == 'RE':
        # Create comparison charts for Room Eligibility
        fig, axs = plt.subplots(1, 2, figsize=(15, 6))
        
        # 1. RE percentage chart
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        axs[0].bar(x - width/2, df['JADE_re_percentage'].values, width, label='JADE', color=IEEE_BLUE)
        axs[0].bar(x + width/2, df['SPADE_re_percentage'].values, width, label='SPADE', color=IEEE_GREY)
        
        axs[0].set_xlabel('Escenario')
        axs[0].set_ylabel('Room Eligibility (%)')
        axs[0].set_title('Comparación de Room Eligibility')
        axs[0].set_xticks(x)
        axs[0].set_xticklabels([s.capitalize() for s in df['Scenario']])
        axs[0].legend()
        
        # Add values on bars
        for i, v in enumerate(df['JADE_re_percentage'].values):
            axs[0].text(i - width/2, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')
            
        for i, v in enumerate(df['SPADE_re_percentage'].values):
            axs[0].text(i + width/2, v + 0.5, f'{v:.2f}%', ha='center', fontweight='bold')
        
        # 2. Satisfaction score chart
        if 'JADE_avg_satisfaction' in df.columns and 'SPADE_avg_satisfaction' in df.columns:
            axs[1].bar(x - width/2, df['JADE_avg_satisfaction'].values, width, label='JADE', color=IEEE_BLUE)
            axs[1].bar(x + width/2, df['SPADE_avg_satisfaction'].values, width, label='SPADE', color=IEEE_GREY)
            
            axs[1].set_xlabel('Escenario')
            axs[1].set_ylabel('Puntuación de Satisfacción Promedio')
            axs[1].set_title('Comparación de Satisfacción con la Programación')
            axs[1].set_xticks(x)
            axs[1].set_xticklabels([s.capitalize() for s in df['Scenario']])
            axs[1].legend()
            
            # Add values on bars
            for i, v in enumerate(df['JADE_avg_satisfaction'].values):
                axs[1].text(i - width/2, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold')
                
            for i, v in enumerate(df['SPADE_avg_satisfaction'].values):
                axs[1].text(i + width/2, v + 0.1, f'{v:.2f}', ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 're_comparison.png'), dpi=300)
        plt.close()
        
        # Create a more comprehensive RE analysis chart
        if 'JADE_limited_flex_events' in df.columns:
            for idx, row in df.iterrows():
                scenario = row['Scenario']
                
                # Extract data for visualization
                jade_total_events = row.get('JADE_total_events', 0)
                jade_limited_flex = row.get('JADE_limited_flex_events', 0)
                jade_re_pct = row.get('JADE_re_percentage', 0)
                
                spade_total_events = row.get('SPADE_total_events', 0)
                spade_limited_flex = row.get('SPADE_limited_flex_events', 0)
                spade_re_pct = row.get('SPADE_re_percentage', 0)
                
                # Create figure with subplots
                fig, axes = plt.subplots(1, 2, figsize=(15, 6))
                
                # JADE Donut Chart
                jade_flex_pct = jade_limited_flex / jade_total_events * 100 if jade_total_events > 0 else 0
                jade_normal_pct = 100 - jade_flex_pct
                
                axes[0].pie([jade_normal_pct, jade_flex_pct], 
                         labels=['Eventos con Flexibilidad Normal', 'Eventos con Flexibilidad Limitada'],
                         colors=[IEEE_BLUE, IEEE_GREY],
                         autopct='%1.1f%%',
                         startangle=90,
                         wedgeprops={'edgecolor': 'w'})
                         
                # Add center circle to create donut chart
                centre_circle = plt.Circle((0,0), 0.5, fc='white')
                axes[0].add_artist(centre_circle)
                axes[0].set_title(f'JADE - Flexibilidad de Eventos ({scenario.capitalize()})')
                axes[0].text(0, 0, f'RE: {jade_re_pct:.1f}%', ha='center', va='center', fontweight='bold')
                
                # SPADE Donut Chart
                spade_flex_pct = spade_limited_flex / spade_total_events * 100 if spade_total_events > 0 else 0
                spade_normal_pct = 100 - spade_flex_pct
                
                axes[1].pie([spade_normal_pct, spade_flex_pct], 
                         labels=['Eventos con Flexibilidad Normal', 'Eventos con Flexibilidad Limitada'],
                         colors=[IEEE_BLUE, IEEE_GREY],
                         autopct='%1.1f%%',
                         startangle=90,
                         wedgeprops={'edgecolor': 'w'})
                         
                # Add center circle to create donut chart
                centre_circle = plt.Circle((0,0), 0.5, fc='white')
                axes[1].add_artist(centre_circle)
                axes[1].set_title(f'SPADE - Flexibilidad de Eventos ({scenario.capitalize()})')
                axes[1].text(0, 0, f'RE: {spade_re_pct:.1f}%', ha='center', va='center', fontweight='bold')
                
                plt.tight_layout()
                plt.savefig(os.path.join(output_dir, f're_flexibility_{scenario}.png'), dpi=300)
                plt.close()

def create_comparative_summary_table(platforms=['JADE', 'SPADE'], scenarios=['full', 'medium', 'small']):
    """Creates a comprehensive summary table comparing all metrics across platforms"""
    
    # Dictionary to store all metrics for each platform and scenario
    all_metrics = {}
    
    # Metrics to collect
    metrics = ['capacity', 'compactness', 'occupancy', 'time_eligibility', 'RE']
    
    # Gather all metrics
    for platform in platforms:
        all_metrics[platform] = {}
        
        for scenario in scenarios:
            all_metrics[platform][scenario] = {}
            
            for metric in metrics:
                data = load_metrics_for_comparison(platform, metric, scenario)
                if data:
                    all_metrics[platform][scenario][metric] = data
    
    # Build comparison data for table
    comparison_rows = []
    
    for scenario in scenarios:
        for platform in platforms:
            row = {
                'Platform': platform,
                'Scenario': scenario
            }
            
            # Add capacity metrics
            if 'capacity' in all_metrics[platform][scenario]:
                cap_data = all_metrics[platform][scenario]['capacity']
                row['Capacidad Óptima (%)'] = cap_data['global_stats']['capacidad_exacta']
                row['Bajo Capacidad (%)'] = cap_data['global_stats']['bajo_capacidad']
            
            # Add occupancy metrics
            if 'occupancy' in all_metrics[platform][scenario]:
                occ_data = all_metrics[platform][scenario]['occupancy']
                row['Ocupación de Salas (%)'] = occ_data['global_stats']['tasa_ocupacion_global']
            
            # Add compactness metrics (average windows)
            if 'compactness' in all_metrics[platform][scenario]:
                comp_data = all_metrics[platform][scenario]['compactness']
                total_ventanas = sum(room['ventanas'] for room in comp_data['estadisticas_por_sala'].values())
                total_rooms = len(comp_data['estadisticas_por_sala'])
                row['Ventanas Promedio/Sala'] = total_ventanas / total_rooms if total_rooms > 0 else 0
            
            # Add time eligibility
            if 'time_eligibility' in all_metrics[platform][scenario]:
                te_data = all_metrics[platform][scenario]['time_eligibility']
                row['TE (%)'] = te_data['te_percentage']
            
            # Add RE metrics
            if 'RE' in all_metrics[platform][scenario]:
                re_data = all_metrics[platform][scenario]['RE']
                row['RE (%)'] = re_data['re_percentage']
                if 'schedule_quality' in re_data and re_data['schedule_quality']:
                    row['Tasa de Programación (%)'] = re_data['schedule_quality'].get('scheduling_rate_percentage', 0)
                    row['Puntuación de Satisfacción'] = re_data['schedule_quality'].get('avg_satisfaction', 0)
            
            comparison_rows.append(row)
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(comparison_rows)
    
    # Save the table
    output_dir = "../scheduling_output/comparative"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    df.to_csv(os.path.join(output_dir, 'comparacion_plataformas_resumen.csv'), index=False)
    
    # Also create a styled HTML table
    styled_table = df.style\
        .background_gradient(subset=['Capacidad Óptima (%)', 'Ocupación de Salas (%)', 'TE (%)', 'RE (%)', 'Puntuación de Satisfacción'], 
                           cmap='Blues')\
        .background_gradient(subset=['Ventanas Promedio/Sala', 'Bajo Capacidad (%)'], 
                           cmap='Blues_r')\
        .format({
            'Capacidad Óptima (%)': '{:.2f}%',
            'Bajo Capacidad (%)': '{:.2f}%',
            'Ocupación de Salas (%)': '{:.2f}%',
            'Ventanas Promedio/Sala': '{:.2f}',
            'TE (%)': '{:.2f}%',
            'RE (%)': '{:.2f}%',
            'Tasa de Programación (%)': '{:.2f}%',
            'Puntuación de Satisfacción': '{:.2f}'
        })\
        .set_caption("Comparación Integral de Plataformas de Scheduling")\
        .set_table_styles([{
            'selector': 'caption',
            'props': [('text-align', 'center'), ('font-size', '16px'), ('font-weight', 'bold')]
        }])
    
    html_path = os.path.join(output_dir, 'comparacion_plataformas_resumen.html')
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(styled_table.to_html())
    
    return df

def create_radar_chart_comparison(df, output_dir):
    """Creates a radar chart comparing overall performance across metrics"""
    
    # We'll create one radar chart per scenario
    scenarios = df['Scenario'].unique()
    
    for scenario in scenarios:
        # Filter data for this scenario
        scenario_data = df[df['Scenario'] == scenario]
        
        # Metrics to use in radar chart
        metrics = [
            'Capacidad Óptima (%)', 
            'Ocupación de Salas (%)',
            'Ventanas Promedio/Sala',
            'TE (%)',
            'RE (%)',
            'Puntuación de Satisfacción'
        ]
        
        metrics_display = [
            'Capacidad\nÓptima', 
            'Ocupación\nde Salas',
            'Compactación\n(inv)',
            'Time-slot\nEligibility',
            'Room\nEligibility',
            'Satisfacción'
        ]
        
        # Scale and normalize the data for radar chart
        max_values = {}
        for metric in metrics:
            if metric in scenario_data.columns:
                max_val = scenario_data[metric].max()
                if metric == 'Ventanas Promedio/Sala':
                    # For windows, lower is better, so we invert the scale
                    max_values[metric] = max_val
                else:
                    max_values[metric] = max_val
        
        # Normalize the metrics (scale 0-1)
        normalized_data = {}
        for platform in ['JADE', 'SPADE']:
            platform_data = scenario_data[scenario_data['Platform'] == platform]
            if len(platform_data) == 0:
                continue
                
            normalized_data[platform] = []
            for metric in metrics:
                if metric in platform_data.columns and metric in max_values and max_values[metric] > 0:
                    value = platform_data[metric].values[0]
                    if metric == 'Ventanas Promedio/Sala':
                        # Invert for windows (lower is better)
                        norm_value = 1 - (value / max_values[metric]) if value > 0 else 1
                    else:
                        # Scale for other metrics (higher is better)
                        norm_value = value / max_values[metric]
                        
                    # Cap at 0-1 range
                    norm_value = max(0, min(1, norm_value))
                    normalized_data[platform].append(norm_value)
                else:
                    normalized_data[platform].append(0)
        
        # Create the radar chart
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, polar=True)
        
        # Set number of metrics as angles
        angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False).tolist()
        angles += angles[:1]  # Close the loop
        
        # Plot platforms
        colors = {'JADE': IEEE_BLUE, 'SPADE': IEEE_GREY}
        linestyles = {'JADE': '-', 'SPADE': '--'}
        markers = {'JADE': 'o', 'SPADE': 's'}
        
        for platform, values in normalized_data.items():
            values += values[:1]  # Close the loop
            ax.plot(angles, values, linestyles[platform], linewidth=2, color=colors[platform], marker=markers[platform], 
                   markersize=8, label=platform)
            ax.fill(angles, values, alpha=0.25, color=colors[platform])
        
        # Set metric labels
        ax.set_xticks(angles[:-1])
        metrics_display_aligned = []
        for metric in metrics_display:
            metrics_display_aligned.append(metric)
        ax.set_xticklabels(metrics_display_aligned)
        
        # Draw axis lines for each angle
        ax.set_rlabel_position(0)
        ax.set_yticks([0.25, 0.5, 0.75, 1])
        ax.set_yticklabels(['0.25', '0.5', '0.75', '1.0'], color='grey')
        ax.set_ylim(0, 1)
        
        # Add gridlines
        ax.grid(True, linestyle='--', alpha=0.6)
        
        # Add legend and title
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.title(f'Comparación de Rendimiento: Escenario {scenario.capitalize()}', fontsize=15, pad=20)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'radar_comparison_{scenario}.png'), dpi=300)
        plt.close()
        
        # Also create a direct comparison bar chart
        create_head_to_head_comparison(scenario_data, metrics, metrics_display, scenario, output_dir)

def create_head_to_head_comparison(scenario_data, metrics, metrics_display, scenario, output_dir):
    """Creates a direct head-to-head bar chart comparison"""
    
    # Prepare data for bar chart
    comparison_data = []
    
    for i, metric in enumerate(metrics):
        if metric not in scenario_data.columns:
            continue
            
        row = {'Metric': metrics_display[i], 'Metric_orig': metric}
        
        for platform in ['JADE', 'SPADE']:
            platform_data = scenario_data[scenario_data['Platform'] == platform]
            if len(platform_data) > 0 and metric in platform_data.columns:
                row[platform] = platform_data[metric].values[0]
        
        comparison_data.append(row)
    
    # Create DataFrame
    comp_df = pd.DataFrame(comparison_data)
    
    # Create bar chart
    plt.figure(figsize=(14, 8))
    
    metrics_display = comp_df['Metric'].values
    x = np.arange(len(metrics_display))
    width = 0.35
    
    jade_bars = plt.bar(x - width/2, comp_df['JADE'].values, width, label='JADE', color=IEEE_BLUE)
    spade_bars = plt.bar(x + width/2, comp_df['SPADE'].values, width, label='SPADE', color=IEEE_GREY)
    
    plt.xlabel('Métrica')
    plt.ylabel('Valor')
    plt.title(f'JADE vs SPADE: Escenario {scenario.capitalize()}')
    plt.xticks(x, metrics_display)
    plt.legend()
    
    # Add value labels
    for i, bar in enumerate(jade_bars):
        metric = comp_df['Metric_orig'].iloc[i]
        value = comp_df['JADE'].iloc[i]
        formatter = '{:.2f}%' if '%' in metric else '{:.2f}'
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                formatter.format(value), ha='center', fontweight='bold')
    
    for i, bar in enumerate(spade_bars):
        metric = comp_df['Metric_orig'].iloc[i]
        value = comp_df['SPADE'].iloc[i]
        formatter = '{:.2f}%' if '%' in metric else '{:.2f}'
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                formatter.format(value), ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'head_to_head_{scenario}.png'), dpi=300)
    plt.close()

def create_3d_comparison_visualization(df, output_dir):
    """Creates an interactive 3D comparison visualization"""
    
    try:
        from mpl_toolkits.mplot3d import Axes3D
        
        # Create a 3D plot for a multi-dimensional comparison
        for scenario in df['Scenario'].unique():
            fig = plt.figure(figsize=(12, 10))
            ax = fig.add_subplot(111, projection='3d')
            
            # Filter data for this scenario
            scenario_data = df[df['Scenario'] == scenario]
            
            # Select 3 metrics for the axes
            x_metric = 'Capacidad Óptima (%)'
            y_metric = 'Ocupación de Salas (%)'
            z_metric = 'TE (%)'
            
            # Extract data for each platform
            for platform in ['JADE', 'SPADE']:
                platform_data = scenario_data[scenario_data['Platform'] == platform]
                if len(platform_data) == 0:
                    continue
                
                x = platform_data[x_metric].values[0]
                y = platform_data[y_metric].values[0]
                z = platform_data[z_metric].values[0]
                
                # Size and color based on other metrics
                if 'RE (%)' in platform_data.columns and 'Puntuación de Satisfacción' in platform_data.columns:
                    re = platform_data['RE (%)'].values[0]
                    satisfaction = platform_data['Puntuación de Satisfacción'].values[0]
                    
                    # Size based on RE
                    size = re * 5
                    
                    # Color based on satisfaction
                    color = IEEE_BLUE if platform == 'JADE' else IEEE_GREY
                    
                    ax.scatter(x, y, z, s=size, c=[color], alpha=0.8, label=f'{platform} (RE: {re:.1f}%)')
                    
                    # Add projections to the walls
                    ax.scatter(x, y, min(ax.get_zlim()), s=size/3, c=[color], alpha=0.3)
                    ax.scatter(x, min(ax.get_ylim()), z, s=size/3, c=[color], alpha=0.3)
                    ax.scatter(min(ax.get_xlim()), y, z, s=size/3, c=[color], alpha=0.3)
                    
                    # Add text labels
                    ax.text(x, y, z, platform, fontsize=10)
            
            # Set labels and title
            ax.set_xlabel(x_metric)
            ax.set_ylabel(y_metric)
            ax.set_zlabel(z_metric)
            ax.set_title(f'Comparación 3D de Métricas: Escenario {scenario.capitalize()}')
            
            # Add legend
            plt.legend()
            
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'3d_comparison_{scenario}.png'), dpi=300)
            plt.close()
    
    except ImportError:
        print("mpl_toolkits.mplot3d not available, skipping 3D visualization")

def create_heatmap_comparison(platforms=['JADE', 'SPADE'], scenarios=['full', 'medium', 'small'], output_dir="../scheduling_output/comparative"):
    """Creates heatmap comparisons for various metrics"""
    
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # For each platform, get room-level metrics
    for platform in platforms:
        # Create occupancy heatmap for each scenario
        for scenario in scenarios:
            occupancy_data = load_metrics_for_comparison(platform, 'occupancy', scenario)
            
            if occupancy_data and 'room_stats' in occupancy_data:
                # Extract room occupancy rates
                room_data = []
                
                for room_code, stats in occupancy_data['room_stats'].items():
                    if 'Tasa de Ocupacion' in stats:
                        room_data.append({
                            'Sala': room_code,
                            'Tasa de Ocupación': stats['Tasa de Ocupacion']
                        })
                
                if room_data:
                    df_room = pd.DataFrame(room_data)
                    
                    # Sort by occupancy rate
                    df_room = df_room.sort_values('Tasa de Ocupación', ascending=False)
                    
                    # Create heatmap
                    plt.figure(figsize=(10, max(8, len(df_room) * 0.25)))
                    
                    # Create a custom colormap (blue to white)
                    cmap = LinearSegmentedColormap.from_list(
                        'custom_cmap', [IEEE_LIGHTBLUE, 'white', IEEE_BLUE], N=100)
                    
                    # Plot heatmap
                    sns.heatmap(df_room[['Tasa de Ocupación']].T, 
                              annot=True, 
                              cmap=cmap,
                              vmin=0, vmax=100,
                              linewidths=.5,
                              fmt='.1f',
                              cbar_kws={'label': 'Tasa de Ocupación (%)'},
                              yticklabels=['Tasa de Ocupación'],
                              xticklabels=df_room['Sala'])
                    
                    plt.title(f'{platform} - Tasa de Ocupación por Sala ({scenario.capitalize()})')
                    plt.tight_layout()
                    plt.savefig(os.path.join(output_dir, f'{platform}_{scenario}_occupancy_heatmap.png'), 
                              dpi=300, bbox_inches='tight')
                    plt.close()

def create_metric_evolution_comparison(df, output_dir):
    """Creates a chart showing how metrics evolve across scenarios"""
    
    # Group the metrics we want to show
    metric_groups = [
        ['TE (%)', 'RE (%)'],
        ['Capacidad Óptima (%)', 'Ocupación de Salas (%)'],
        ['Ventanas Promedio/Sala', 'Puntuación de Satisfacción']
    ]
    
    fig, axes = plt.subplots(len(metric_groups), 1, figsize=(12, 15))
    
    # For each group of metrics
    for i, metrics in enumerate(metric_groups):
        ax = axes[i]
        
        # Set up X axis (scenarios)
        scenarios = sorted(df['Scenario'].unique(), key=lambda x: 
                        {'small': 0, 'medium': 1, 'full': 2}.get(x, 3))
        x = np.arange(len(scenarios))
        
        # Set width and offset for bars
        width = 0.15
        offsets = np.linspace(-0.3, 0.3, len(metrics) * 2)
        
        # List to store legend handles
        legend_handles = []
        
        # For each metric in this group
        for j, metric in enumerate(metrics):
            # For each platform
            for k, platform in enumerate(['JADE', 'SPADE']):
                # Filter data
                platform_data = []
                for scenario in scenarios:
                    scenario_df = df[(df['Scenario'] == scenario) & (df['Platform'] == platform)]
                    if len(scenario_df) > 0 and metric in scenario_df.columns:
                        platform_data.append(scenario_df[metric].values[0])
                    else:
                        platform_data.append(0)
                
                # Calculate offset for this bar
                offset_idx = j * 2 + k
                offset = offsets[offset_idx]
                
                # Color based on platform
                color = IEEE_BLUE if platform == 'JADE' else IEEE_GREY
                
                # Plot the bar
                bars = ax.bar(x + offset, platform_data, width, 
                            label=f'{platform} - {metric}',
                            color=color, alpha=0.8 if k == 0 else 0.6)
                
                # Add value labels
                for idx, bar in enumerate(bars):
                    value = platform_data[idx]
                    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                            f'{value:.1f}', ha='center', fontsize=8)
                
                # Store for legend
                legend_handles.append(bars)
        
        # Set labels and title
        ax.set_xlabel('Escenario')
        ax.set_ylabel('Valor')
        ax.set_title(f'Evolución de Métricas: {", ".join(metrics)}')
        ax.set_xticks(x)
        ax.set_xticklabels([s.capitalize() for s in scenarios])
        
        # Add legend
        ax.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'metric_evolution.png'), dpi=300)
    plt.close()

def generate_all_comparisons():
    """Generates all comparison charts and tables"""
    output_dir = "../scheduling_output/comparative"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Generate comparison for each metric
    metrics = ['capacity', 'compactness', 'occupancy', 'time_eligibility', 'RE']
    
    for metric in metrics:
        print(f"Generando comparación para {metric}...")
        df = create_comparison_dataframe(metric)
        df.to_csv(os.path.join(output_dir, f'{metric}_comparison.csv'), index=False)
        create_comparative_charts(metric, df, output_dir)
    
    # Generate comprehensive summary table
    print("Generando resumen comparativo integral...")
    summary_df = create_comparative_summary_table()
    
    # Create summary visualization
    print("Generando gráfico radar...")
    create_radar_chart_comparison(summary_df, output_dir)
    
    # Create 3D visualization
    print("Generando visualización 3D...")
    create_3d_comparison_visualization(summary_df, output_dir)
    
    # Create heatmaps
    print("Generando mapas de calor...")
    create_heatmap_comparison(output_dir=output_dir)
    
    # Create metric evolution comparison
    print("Generando gráfico de evolución de métricas...")
    create_metric_evolution_comparison(summary_df, output_dir)
    
    print(f"Todas las comparaciones han sido generadas y guardadas en {output_dir}")
    return summary_df

# If run as main script
if __name__ == "__main__":
    # Generate all comparisons
    summary_df = generate_all_comparisons()
    print("Comparación completa. Resumen de resultados:")
    print(summary_df[['Platform', 'Scenario', 'Capacidad Óptima (%)', 'Ocupación de Salas (%)', 'TE (%)', 'RE (%)', 'Puntuación de Satisfacción']])

Generando comparación para capacity...
Generando comparación para compactness...
Generando comparación para occupancy...
Generando comparación para time_eligibility...
Generando comparación para RE...
Generando resumen comparativo integral...
Generando gráfico radar...
Generando visualización 3D...
Generando mapas de calor...
Generando gráfico de evolución de métricas...
Todas las comparaciones han sido generadas y guardadas en ../scheduling_output/comparative
Comparación completa. Resumen de resultados:
  Platform Scenario  Capacidad Óptima (%)  Ocupación de Salas (%)     TE (%)  \
0     JADE     full             31.149144               62.252664  90.163540   
1    SPADE     full             32.093933               62.222222  90.252742   
2     JADE   medium             31.416838               54.111111  89.068462   
3    SPADE   medium             30.234934               54.388889  88.956571   
4     JADE    small             36.000000               48.148148  87.547893   
5    SPADE